# Test reading in `dim_monthly_ntd_ridership_with_adjustments` from warehouse

In [1]:
import pandas as pd
from calitp_data_analysis.tables import tbls
from siuba import _, collect, count, filter, show_query


In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [3]:
monthly_ridership = (
    tbls.mart_ntd.dim_monthly_ntd_ridership_with_adjustments() >> collect()
)

In [41]:
# reading in old monthly ridership parquet for comparison
old_march_2024_ridership = pd.read_parquet("gs://calitp-analytics-data/data-analyses/ntd/ntd_monthly_ridership_2024_March.parquet")

In [4]:
# what is in this table?
# all states
display(
    monthly_ridership.shape,
    # monthly_ridership.info,
    monthly_ridership.head(3),
)

(613184, 22)

,uza_name,uace_cd,_dt,ts,ntd_id,year,reporter_type,agency,mode_type_of_service_status,mode,mode_full_name,service_type,_3_mode,tos,legacy_ntd_id,period_year_month,period_year,period_month,upt,vrm,vrh,voms
0,"Miami--Fort Lauderdale, FL",56602,2024-07-01,2024-07-01 21:44:31.635354+00:00,40034,2024,Full Reporter,County of Miami-Dade,Inactive,AG,Automated Guideway,Fixed Route,Rail,DO,4034,2016-01,2016,1,0,0,0,0
1,"Miami--Fort Lauderdale, FL",56602,2024-07-01,2024-07-01 21:44:31.635354+00:00,40034,2024,Full Reporter,County of Miami-Dade,Inactive,AG,Automated Guideway,Fixed Route,Rail,DO,4034,2018-07,2018,7,0,0,0,0
2,"Miami--Fort Lauderdale, FL",56602,2024-07-01,2024-07-01 21:44:31.635354+00:00,40034,2024,Full Reporter,County of Miami-Dade,Inactive,AG,Automated Guideway,Fixed Route,Rail,DO,4034,2013-04,2013,4,0,0,0,0


In [5]:
print(monthly_ridership.dtypes)

uza_name                                    object
uace_cd                                     object
_dt                                         object
ts                             datetime64[ns, UTC]
ntd_id                                      object
year                                         int64
reporter_type                               object
agency                                      object
mode_type_of_service_status                 object
mode                                        object
mode_full_name                              object
service_type                                object
_3_mode                                     object
tos                                         object
legacy_ntd_id                               object
period_year_month                           object
period_year                                 object
period_month                                object
upt                                          int64
vrm                            

In [6]:
# are there NaN values in uza name? YES
monthly_ridership["uza_name"].isna().sum()

1608

In [7]:
# fill the NaN values with something
monthly_ridership["uza_name"].fillna("none", inplace=True)

In [8]:
# double check for na
monthly_ridership["uza_name"].isna().sum()

0

In [9]:
# can i now filter by uza CA, becaue there are zero NaN values? YES
uza_ca = monthly_ridership[monthly_ridership["uza_name"].str.contains(", CA")]
display(
    uza_ca.shape,
    uza_ca["uza_name"].value_counts()
)

(91924, 22)

Los Angeles--Long Beach--Anaheim, CA             19296
San Francisco--Oakland, CA                       10720
Sacramento, CA                                    8308
San Diego, CA                                     6164
Riverside--San Bernardino, CA                     3752
Stockton, CA                                      2948
Vallejo, CA                                       2144
Santa Rosa, CA                                    1876
Lake Tahoe, CA-NV                                 1876
Modesto, CA                                       1876
Seaside--Monterey--Pacific Grove, CA              1876
San Luis Obispo, CA                               1608
Palmdale--Lancaster, CA                           1608
Oxnard--San Buenaventura (Ventura), CA            1340
San Jose, CA                                      1340
Santa Cruz, CA                                    1340
Visalia, CA                                       1340
Hanford, CA                                       1340
Indio--Pal

In [28]:
# where are the UTP values? the UTP page for this table used to be very wide
# look at a specific agency by ntd id, LA Metro == "9154"
# what is the dtype of ntd_id... its object, so use str.contrains
la_metro = monthly_ridership[monthly_ridership["ntd_id"] == "90154"]
display(la_metro.shape, uza_ca.shape)

(1876, 22)

(91924, 22)

In [11]:
# filter la_metro data to only see current and previous year (2024 and 2023)

la_2324 = la_metro[la_metro["period_year"].isin(["2023", "2024"])]

In [12]:
# does this 23/24 data match whats in the NTD report? YES
# confirm that data is now long instead of wide
la_2324[la_2324["period_year_month"] == "2024-03"][
    ["period_year_month", "mode", "tos", "upt"]
].sort_values(by="period_year_month", ascending=True)

,period_year_month,mode,tos,upt
286578,2024-03,DR,DO,62986
321738,2024-03,HR,DO,1874721
331816,2024-03,LR,DO,3842909
533491,2024-03,MB,PT,929537
533501,2024-03,MB,DO,18852906
556718,2024-03,RB,DO,380625
607310,2024-03,VP,PT,125149


# how to modify the `produce_ntd_monthly_ridership_by_rtpa` function
step-by-step

In [13]:
# 1. reading in the data
# intead of reading in the data via an excel doc by URL, intead will query the warehouse directly

#GOOD TO GO
full_upt = (tbls.mart_ntd.dim_monthly_ntd_ridership_with_adjustments() >> collect()).rename(columns = {"mode_type_of_service_status": "Status"})


In [63]:
display(
    full_upt.dtypes,

)

uza_name                          object
uace_cd                           object
_dt                               object
ts                   datetime64[ns, UTC]
ntd_id                            object
year                               int64
reporter_type                     object
agency                            object
Status                            object
mode                              object
mode_full_name                    object
service_type                      object
_3_mode                           object
tos                               object
legacy_ntd_id                     object
period_year_month                 object
period_year                       object
period_month                      object
upt                                int64
vrm                                int64
vrh                                int64
voms                               int64
dtype: object

In [51]:
#1.1 setting `ntd_id` to str

#GOOD TO GO!
full_upt["ntd_id"] = full_upt["ntd_id"].astype(str)

In [61]:
display(
    #full_upt.shape,
    full_upt["ntd_id"].dtypes,
    full_upt.dtypes
)

dtype('O')

uza_name                          object
uace_cd                           object
_dt                               object
ts                   datetime64[ns, UTC]
ntd_id                            object
year                               int64
reporter_type                     object
agency                            object
Status                            object
mode                              object
mode_full_name                    object
service_type                      object
_3_mode                           object
tos                               object
legacy_ntd_id                     object
period_year_month                 object
period_year                       object
period_month                      object
upt                                int64
vrm                                int64
vrh                                int64
voms                               int64
dtype: object

In [64]:
#2 filter df to get rid of na rows
# GOOD TO GO!
full_upt = full_upt[full_upt.agency.notna()].reset_index(drop=True)

In [69]:
full_upt["agency"].isna().sum()

0

In [17]:
ca = full_upt[(full_upt["uza_name"].str.contains(", CA")) & 
            (full_upt.agency.notna())].reset_index(drop=True)

In [18]:
ca.shape

(91924, 22)

In [19]:
crosswalk = pd.read_csv(
        f"gs://calitp-analytics-data/data-analyses/ntd/ntd_id_rtpa_crosswalk.csv", 
        dtype = {"NTD ID": "str"}
    #have to rename NTD ID col to match the dim table
    ).rename(columns={"NTD ID": "ntd_id"})
    
crosswalk.columns

Index(['ntd_id', 'Legacy NTD ID', 'Agency', 'UZA Name', 'RTPA_open_data',
       'RTPA'],
      dtype='object')

In [20]:
df = pd.merge(
        ca,
        # Merging on too many columns can create problems 
        # because csvs and dtypes aren't stable / consistent 
        # for NTD ID, Legacy NTD ID, and UZA
        crosswalk[["ntd_id", "RTPA"]],
        #change on = value to ntd_id
        on = "ntd_id",
        how = "left",
        indicator = True
    )
    
print(df._merge.value_counts())

both          91924
left_only         0
right_only        0
Name: _merge, dtype: int64


In [21]:
# updated to match new year-month format 

def add_change_columns(
    df: pd.DataFrame,
    year: int,
    month: int
) -> pd.DataFrame:
    """
    """    
    ntd_month_col = f"{year}-{month}"
    prior_year_col = f"{int(year)-1}-{month}"
        
    df[f"change_1yr_{ntd_month_col}"] = df[ntd_month_col] - df[prior_year_col]
    df = get_percent_change(df, ntd_month_col, prior_year_col)
    
    return df

In [35]:
from shared_utils.rt_dates import MONTH_DICT
from update_vars import GCS_FILE_PATH, NTD_MODES, NTD_TOS, YEAR, MONTH

# final updated  `produce_ntd_monthly_ridership_by_rtpa` function.
# may not need the upt_url

def produce_ntd_monthly_ridership_by_rtpa(
    #upt_url: str,
    year: int,
    month: str
) -> pd.DataFrame:
    """
    Import NTD data from url, filter to CA, 
    merge in crosswalk, and save individual csvs.
    """
    full_upt = (tbls.mart_ntd.dim_monthly_ntd_ridership_with_adjustments() >> collect()
               ).rename(columns = {"mode_type_of_service_status": "Status"})
    
    full_upt = full_upt[full_upt.agency.notna()].reset_index(drop=True)
    #full_upt.to_parquet(
    #    f"{GCS_FILE_PATH}ntd_monthly_ridership_{year}_{month}.parquet"
    #)
    ca = full_upt[(full_upt["uza_name"].str.contains(", CA")) & 
            (full_upt.agency.notna())].reset_index(drop=True)
    
    crosswalk = pd.read_csv(
        f"gs://calitp-analytics-data/data-analyses/ntd/ntd_id_rtpa_crosswalk.csv", 
        dtype = {"NTD ID": "str"}
    #have to rename NTD ID col to match the dim table
    ).rename(columns={"NTD ID": "ntd_id"})
    
    df = pd.merge(
        ca,
        # Merging on too many columns can create problems 
        # because csvs and dtypes aren't stable / consistent 
        # for NTD ID, Legacy NTD ID, and UZA
        crosswalk[["ntd_id", "RTPA"]],
        #change on = value to ntd_id
        on = "ntd_id",
        how = "left",
        indicator = True
    )
    
    print(df._merge.value_counts())
    
    # Good, everything merged, as we want
    if len(df[df._merge=="left_only"]) > 0:
        raise ValueError("There are unmerged rows to crosswalk")
        
    # Add new columns    
    reversed_months = {v:k for k, v in MONTH_DICT.items()}
    
    for m in range(1, reversed_months[month] + 1):
        df = add_change_columns(df, year, m)
    
    df = df.assign(
        Mode_full = df.Mode.map(NTD_MODES),
        TOS_full = df.TOS.map(NTD_TOS)
    )
    
    return df


In [31]:
ca["period_year_month"].value_counts().head()

2007-06    343
2016-05    343
2017-03    343
2015-12    343
2005-02    343
Name: period_year_month, dtype: int64

In [32]:
# what is the most recent period_year_month? as of 7/2/2024\
# should expect to see values up tp May 2024
display(
    ca[ca["period_year_month"] == "2024-04"].shape,
    # as of 7/2, zero rows for May 2024. but expect ntd to update that data soon.
    # 7/15, still no values for May 2024
    ca[ca["period_year_month"] == "2024-05"].shape
)

(343, 22)

(0, 22)

---
## Do I need `reserved_months` anymore?
RE: `produce_ntd_monthly_ridership_by_rtpa` function.


Since the new table uses a `yyyy-mm` format now. i dont think i need the revesre months anymore. what else needs to change to the the function working again.

In [33]:
# do i need reversed_months anymore?

reversed_months = {v:k for k, v in MONTH_DICT.items()}

In [38]:
# what MONTH_DICT looks like initially.
MONTH_DICT

{1: 'January',
 2: 'February',
 3: 'March',
 4: 'April',
 5: 'May',
 6: 'June',
 7: 'July',
 8: 'August',
 9: 'September',
 10: 'October',
 11: 'November',
 12: 'December'}

In [34]:
# like the name implies, reverses the key values pairs. 
reversed_months

{'January': 1,
 'February': 2,
 'March': 3,
 'April': 4,
 'May': 5,
 'June': 6,
 'July': 7,
 'August': 8,
 'September': 9,
 'October': 10,
 'November': 11,
 'December': 12}

In [43]:
old_march_2024_ridership.head()

,NTD ID,Legacy NTD ID,Agency,Status,Reporter Type,UACE CD,UZA Name,Mode,TOS,3 Mode,1/2002,2/2002,3/2002,4/2002,5/2002,6/2002,7/2002,8/2002,9/2002,10/2002,11/2002,12/2002,1/2003,2/2003,3/2003,4/2003,5/2003,6/2003,7/2003,8/2003,9/2003,10/2003,11/2003,12/2003,1/2004,2/2004,3/2004,4/2004,5/2004,6/2004,7/2004,8/2004,9/2004,10/2004,11/2004,12/2004,1/2005,2/2005,3/2005,4/2005,5/2005,6/2005,7/2005,8/2005,9/2005,10/2005,11/2005,12/2005,1/2006,2/2006,3/2006,4/2006,5/2006,6/2006,7/2006,8/2006,9/2006,10/2006,11/2006,12/2006,1/2007,2/2007,3/2007,4/2007,5/2007,6/2007,7/2007,8/2007,9/2007,10/2007,11/2007,12/2007,1/2008,2/2008,3/2008,4/2008,5/2008,6/2008,7/2008,8/2008,9/2008,10/2008,11/2008,12/2008,1/2009,2/2009,3/2009,4/2009,5/2009,6/2009,7/2009,8/2009,9/2009,10/2009,11/2009,12/2009,1/2010,2/2010,3/2010,4/2010,5/2010,6/2010,7/2010,8/2010,9/2010,10/2010,11/2010,12/2010,1/2011,2/2011,3/2011,4/2011,5/2011,6/2011,7/2011,8/2011,9/2011,10/2011,11/2011,12/2011,1/2012,2/2012,3/2012,4/2012,5/2012,6/2012,7/2012,8/2012,9/2012,10/2012,11/2012,12/2012,1/2013,2/2013,3/2013,4/2013,5/2013,6/2013,7/2013,8/2013,9/2013,10/2013,11/2013,12/2013,1/2014,2/2014,3/2014,4/2014,5/2014,6/2014,7/2014,8/2014,9/2014,10/2014,11/2014,12/2014,1/2015,2/2015,3/2015,4/2015,5/2015,6/2015,7/2015,8/2015,9/2015,10/2015,11/2015,12/2015,1/2016,2/2016,3/2016,4/2016,5/2016,6/2016,7/2016,8/2016,9/2016,10/2016,11/2016,12/2016,1/2017,2/2017,3/2017,4/2017,5/2017,6/2017,7/2017,8/2017,9/2017,10/2017,11/2017,12/2017,1/2018,2/2018,3/2018,4/2018,5/2018,6/2018,7/2018,8/2018,9/2018,10/2018,11/2018,12/2018,1/2019,2/2019,3/2019,4/2019,5/2019,6/2019,7/2019,8/2019,9/2019,10/2019,11/2019,12/2019,1/2020,2/2020,3/2020,4/2020,5/2020,6/2020,7/2020,8/2020,9/2020,10/2020,11/2020,12/2020,1/2021,2/2021,3/2021,4/2021,5/2021,6/2021,7/2021,8/2021,9/2021,10/2021,11/2021,12/2021,1/2022,2/2022,3/2022,4/2022,5/2022,6/2022,7/2022,8/2022,9/2022,10/2022,11/2022,12/2022,1/2023,2/2023,3/2023,4/2023,5/2023,6/2023,7/2023,8/2023,9/2023,10/2023,11/2023,12/2023,1/2024,2/2024,3/2024
0,00001,0001,King County Department of Metro Transit,Active,Full Reporter,80389.0,"Seattle--Tacoma, WA",DR,PT,Bus,135144.0,127378.0,136030.0,142204.0,144697.0,131833.0,137547.0,136915.0,132111.0,150074.0,127835.0,114857.0,137888.0,126339.0,140920.0,144048.0,142725.0,135650.0,143660.0,136752.0,136291.0,152853.0,127150.0,131877.0,132563.0,136857.0,156728.0,151997.0,147491.0,150217.0,146303.0,145459.0,142994.0,147128.0,121962.0,127022.0,143393.0,139615.0,157100.0,152143.0,164149.0,159630.0,156130.0,167625.0,156514.0,160198.0,150451.0,147735.0,150003.0,141801.0,163541.0,152336.0,163392.0,162033.0,162413.0,176618.0,165721.0,168266.0,143859.0,136760.0,149158.0,147755.0,164726.0,163174.0,181919.0,172644.0,176029.0,185516.0,162255.0,179185.0,159051.0,150324.0,95701.0,94361.0,97490.0,102270.0,101764.0,96095.0,101836.0,96210.0,95551.0,108679.0,91534.0,94585.0,94608.0,88625.0,97448.0,98250.0,94046.0,94180.0,97728.0,95537.0,95663.0,103539.0,90144.0,92684.0,93494.0,91935.0,106040.0,103020.0,98987.0,99904.0,97480.0,99207.0,97192.0,104089.0,89891.0,95936.0,97988.0,91581.0,108150.0,101288.0,102906.0,100710.0,94743.0,101592.0,96300.0,99660.0,93937.0,88246.0,81322.0,93458.0,99497.0,95576.0,101273.0,92881.0,93252.0,97683.0,88042.0,100203.0,91161.0,85044.0,94010.0,88791.0,95012.0,97481.0,99526.0,89967.0,94243.0,93538.0,86747.0,98441.0,85117.0,80482.0,89760.0,80405.0,87426.0,91219.0,88900.0,83377.0,84799.0,82018.0,80946.0,90614.0,76389.0,76861.0,78761.0,75234.0,80757.0,79109.0,77368.0,76695.0,75763.0,73699.0,71205.0,76889.0,67696.0,69450.0,69003.0,72120.0,77530.0,75395.0,74815.0,73367.0,69480.0,76378.0,71898.0,74847.0,70426.0,65517.0,69528.0,63363.0,76764.0,71981.0,75959.0,72475.0,69215.0,74755.0,67205.0,75435.0,70504.0,66971.0,73712.0,67250.0,74959.0,74077.0,77958.0,72783.0,73821.0,77780.0,68824.0,79872.0,72742.0,69534.0,75357.0,52494.0,74996.0,77393.0,79347.0,73323.0,78161.0,78350.0,73259.0,82385.0,72116.0,70734.0,73573.0,71893.0,40217.0,19769.0,22043.0,25108.0,36591

In [44]:
# for month, use full month name
produce_ntd_monthly_ridership_by_rtpa(year=2024, month="March")

both          91924
left_only         0
right_only        0
Name: _merge, dtype: int64


KeyError: '2024-1'